Week: 11 Decision Tree

Write a program to demonstrate the working of the decision tree based ID3, C4.5, and CART algorithms without using scikit-learn library. 
Use an appropriate data set (following) for building the decision tree and apply this knowledge to classify a new sample.

In [10]:
import pandas as pd
import numpy as np
from math import sqrt, log2

In [3]:
df = pd.read_csv('/home/AIML_Student/Documents/B34_aiml/Lab11/data.csv')

In [4]:
df

,Day,Outlook,Temp,Humidity,Wind,Decision
0,1,Sunny,85,85,Weak,No
1,2,Sunny,80,90,Strong,No
2,3,Overcast,83,78,Weak,Yes
3,4,Rain,70,96,Weak,Yes
4,5,Rain,68,80,Weak,Yes
5,6,Rain,65,70,Strong,No
6,7,Overcast,64,65,Strong,Yes
7,8,Sunny,72,95,Weak,No
8,9,Sunny,69,70,Weak,Yes
9,10,Rain,75,80,Weak,Yes


In [7]:
cols = df.columns

In [95]:
x = df.iloc[:, :-1]
y = df.iloc[:, -1]

x = x.drop([cols[0]], axis = 1)
print(x)


     Outlook  Temp  Humidity    Wind
0      Sunny    85        85    Weak
1      Sunny    80        90  Strong
2   Overcast    83        78    Weak
3       Rain    70        96    Weak
4       Rain    68        80    Weak
5       Rain    65        70  Strong
6   Overcast    64        65  Strong
7      Sunny    72        95    Weak
8      Sunny    69        70    Weak
9       Rain    75        80    Weak
10     Sunny    75        70  Strong
11  Overcast    72        90  Strong
12  Overcast    81        75    Weak
13      Rain    71        80  Strong


In [21]:
#calculating Entropy: 

"""
Entropy(s)= -P(yes)log2 P(yes)- P(no) log2 P(no)

Where,
    S= Total number of samples
    P(yes)= probability of yes
    P(no)= probability of no
    
Note that the more uniform is the probability distribution, 
the greater is its information.

When entropy is zero the sample is completely homogeneous, meaning that 
each instance belongs to the same class and entropy is one when the 
sample is equally divided between different classes.

i.e. 
if S is (0.5+, 0.5-) then Entropy(S) is 1, if S is (0.67+, 0.33-) 
then Entropy(S) is 0.92, if P is (1+, 0-) then Entropy(S) is 0.
"""
size = df.shape
print(size)

n = size[0] #number of rows


"""

#for Target Variable:

yes_count = df[cols[-1]].value_counts()['Yes']

no_count = n - yes_count

P_yes = yes_count/n
P_no = no_count/n

Entropy = -(P_yes*log2(P_yes))-(P_no*log2(P_no))

print("Entropy of given dataset: ", Entropy)

"""


(14, 6)


'\n\n#for Target Variable:\n\nyes_count = df[cols[-1]].value_counts()[\'Yes\']\n\nno_count = n - yes_count\n\nP_yes = yes_count/n\nP_no = no_count/n\n\nEntropy = -(P_yes*log2(P_yes))-(P_no*log2(P_no))\n\nprint("Entropy of given dataset: ", Entropy)\n\n'

In [22]:
unique = df[cols[-1]].unique()
unique

array(['No', 'Yes'], dtype=object)

In [63]:
#for attributes:

Entropy = []
total_Entropy = 0

for i in cols:
    e = 0
    if i != 'Day':
        feature = df[i]
        unique = df[i].unique()
        for j in unique:
            P = feature.value_counts()[j]
            P /= n
            e -= P*log2(P)
        Entropy.append(e)
        total_Entropy += e
        print(f"Entropy of col {i} is: {Entropy[-1]}")
    
print(f"\nTotal entropy of dataset: {total_Entropy}")

Entropy of col Outlook is: 1.5774062828523454
Entropy of col Temp is: 3.521640636343319
Entropy of col Humidity is: 2.9852281360342516
Entropy of col Wind is: 0.9852281360342515
Entropy of col Decision is: 0.9402859586706311

Total entropy of dataset: 10.0097891499348


In [37]:
print(cols)

Index(['Day', 'Outlook', 'Temp', 'Humidity', 'Wind', 'Decision'], dtype='object')


In [42]:
test = df.loc[(df[cols[1]] == 'Sunny') & (df[cols[-1]]== 'Yes')]
test

,Day,Outlook,Temp,Humidity,Wind,Decision
8,9,Sunny,69,70,Weak,Yes
10,11,Sunny,75,70,Strong,Yes


In [43]:
len(test)

2

In [64]:
#calculating Information Gain for each attribute

Gain = []

for i in range(1, len(cols)-1):
    g=0
    e = 0
    unique = df[cols[i]].unique()
    for j in unique:
        P = df[cols[i]].value_counts()[j]
        P_Yes = len(df.loc[(df[cols[i]] == j) & (df[cols[-1]]== 'Yes')])/n
        P_no = len(df.loc[(df[cols[i]] == j) & (df[cols[-1]]== 'No')])/n

        #entropy for each unique value
        if(P_no == 0):
            e = -(P_yes*log2(P_yes))
        elif P_yes == 0:
            e = -(P_no*log2(P_no))
        else:
            e = -(P_yes*log2(P_yes))-(P_no*log2(P_no)) 
        g = g+   (P_yes * e) + (P_no * e)
        
    g = Entropy[-1] - g
    Gain.append(g)
    print(f"\nInformation gain of {cols[i]} = {Gain[-1]}")


Information gain of Outlook = -0.7196515545466315

Information gain of Temp = -3.338458813618308

Information gain of Humidity = -2.5481757997942465

Information gain of Wind = -0.4562238832719445


In [ ]:
#implementing using ID3

Steps to making Decision Tree (ID3):

a) Take the Entire dataset as an input.

b) Calculate the Entropy of the target variable, As well as the predictor attributes

c) Calculate the information gain of all attributes.

d) Choose the attribute with the highest information gain as the Root Node

e) Repeat the same procedure on every branch until the decision node of each branch is finalized.


In [59]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import copy

dataset = df
X = dataset.iloc[:, 1:].values
# print(X)
attribute = ['outlook', 'temp', 'humidity', 'wind']


class Node(object):
    def __init__(self):
        self.value = None
        self.decision = None
        self.childs = None


def findEntropy(data, rows):
    yes = 0
    no = 0
    ans = -1
    idx = len(data[0]) - 1
    entropy = 0
    for i in rows:
        if data[i][idx] == 'Yes':
            yes = yes + 1
        else:
            no = no + 1

    x = yes/(yes+no)
    y = no/(yes+no)
    if x != 0 and y != 0:
        entropy = -1 * (x*math.log2(x) + y*math.log2(y))
    if x == 1:
        ans = 1
    if y == 1:
        ans = 0
    return entropy, ans


def findMaxGain(data, rows, columns):
    maxGain = 0
    retidx = -1
    entropy, ans = findEntropy(data, rows)
    if entropy == 0:
        """if ans == 1:
            print("Yes")
        else:
            print("No")"""
        return maxGain, retidx, ans

    for j in columns:
        mydict = {}
        idx = j
        for i in rows:
            key = data[i][idx]
            if key not in mydict:
                mydict[key] = 1
            else:
                mydict[key] = mydict[key] + 1
        gain = entropy

        # print(mydict)
        for key in mydict:
            yes = 0
            no = 0
            for k in rows:
                if data[k][j] == key:
                    if data[k][-1] == 'Yes':
                        yes = yes + 1
                    else:
                        no = no + 1
            # print(yes, no)
            x = yes/(yes+no)
            y = no/(yes+no)
            # print(x, y)
            if x != 0 and y != 0:
                gain += (mydict[key] * (x*math.log2(x) + y*math.log2(y)))/14
        # print(gain)
        if gain > maxGain:
            # print("hello")
            maxGain = gain
            retidx = j

    return maxGain, retidx, ans


def buildTree(data, rows, columns):

    maxGain, idx, ans = findMaxGain(X, rows, columns)
    root = Node()
    root.childs = []
    # print(maxGain
    #
    # )
    if maxGain == 0:
        if ans == 1:
            root.value = 'Yes'
        else:
            root.value = 'No'
        return root

    root.value = attribute[idx]
    mydict = {}
    for i in rows:
        key = data[i][idx]
        if key not in mydict:
            mydict[key] = 1
        else:
            mydict[key] += 1

    newcolumns = copy.deepcopy(columns)
    newcolumns.remove(idx)
    for key in mydict:
        newrows = []
        for i in rows:
            if data[i][idx] == key:
                newrows.append(i)
        # print(newrows)
        temp = buildTree(data, newrows, newcolumns)
        temp.decision = key
        root.childs.append(temp)
    return root


def traverse(root):
    print(root.decision)
    print(root.value)

    n = len(root.childs)
    if n > 0:
        for i in range(0, n):
            traverse(root.childs[i])


def calculate():
    rows = [i for i in range(0, 14)]
    columns = [i for i in range(0, 4)]
    root = buildTree(X, rows, columns)
    root.decision = 'Start'
    traverse(root)


calculate()

Start
temp
85
No
80
No
83
Yes
70
Yes
68
Yes
65
No
64
Yes
72
outlook
Sunny
No
Overcast
Yes
69
Yes
75
Yes
81
Yes
71
No


In [ ]:
#implement C45 algorithm

In [102]:
def entropy(data, target_attribute):
    value_counts = data[target_attribute].value_counts()
    total = len(data)
    entropy = 0
    for value_count in value_counts:
        probability = value_count / total
        entropy -= probability * math.log2(probability)
    return entropy
    
def information_gain(data, attribute, target_attribute):
    total_entropy = entropy(data, target_attribute)
    values, counts = np.unique(data[attribute], return_counts=True)
    weighted_entropy = 0
    for value, count in zip(values, counts):
        subset = data[data[attribute] == value]
        weighted_entropy += (count / len(data)) * entropy(subset, target_attribute)
    return total_entropy - weighted_entropy

target_attribute = 'Decision'

predictor_attributes = [col for col in data.columns if col != target_attribute]

target_entropy = entropy(data, target_attribute)
print("Entropy of the target variable (Decision):", target_entropy)

for attribute in predictor_attributes:
    entropy_value = entropy(data, attribute)
    print(f"Entropy of {attribute}: {entropy_value}")
    
print(' ')
information_gains = {}
for attribute in predictor_attributes:
    gain = information_gain(data, attribute, target_attribute)
    information_gains[attribute] = gain

for attribute, gain in information_gains.items():
    print(f"Information Gain for {attribute}: {gain}")

Entropy of the target variable (Decision): 0.9402859586706311
Entropy of Day: 3.8073549220576055
Entropy of Outlook: 1.5774062828523454
Entropy of Temp: 3.521640636343319
Entropy of Humidity: 2.9852281360342525
Entropy of Wind: 0.9852281360342515
 
Information Gain for Day: 0.9402859586706311
Information Gain for Outlook: 0.24674981977443933
Information Gain for Temp: 0.7974288158134883
Information Gain for Humidity: 0.4038734583615642
Information Gain for Wind: 0.04812703040826949


In [106]:
class Node:
    def __init__(self, attribute=None, value=None, result=None):
        self.attribute = attribute
        self.value = value
        self.result = result
        self.children = {}

def label_encode(data):
    labels = {}
    for column in data.columns:
        label_map = {value: index for index, value in enumerate(data[column].unique())}
        data[column] = data[column].map(label_map)
        labels[column] = label_map
    return data, labels

"""
def entropy(data, target_attribute):
    value_counts = data[target_attribute].value_counts()
    total = len(data)
    entropy = 0
    for value_count in value_counts:
        probability = value_count / total
        entropy -= probability * math.log2(probability)
    return entropy

def id3(data, attributes, target_attribute):
    pass

def classify(tree, sample):
    pass

df = df.copy()
data, attribute_mappings = label_encode(df)

attributes = data.columns[1:-1]
target_attribute = 'Decision'

tree = id3(data.to_numpy(), attributes, target_attribute)

new_sample = pd.Series({"Outlook": "Sunny", "Wind": "Weak", "Temp": 85, "Humidity": 85})
new_sample = new_sample.map(attribute_mappings)
result = classify(tree, new_sample)
if result is not None:
    predicted_class = list(attribute_mappings[target_attribute].keys())[list(attribute_mappings[target_attribute].values()).index(result)]
    print("Predicted class:", predicted_class)
else:
    print("Unable to classify the sample.")
result = classify(tree, new_sample)
print("Predicted decision:", result)"""


Unable to classify the sample.
Predicted decision: None


In [104]:
def c45(data, attributes, target_attribute, parent_data, parent_value=None):
    if len(np.unique(data[target_attribute])) == 1:
        return Node(result=data[target_attribute].iloc[0])

    if len(attributes) == 0:
        most_common = data[target_attribute].value_counts().idxmax()
        return Node(result=most_common)

    best_attribute = max(attributes, key=lambda a: information_gain(data, a, target_attribute))
    tree = Node(attribute=best_attribute)

    values = np.unique(data[best_attribute])
    for value in values:
        subset = data[data[best_attribute] == value]
        if len(subset) == 0:
            most_common = data[target_attribute].value_counts().idxmax()
            tree.children[value] = Node(result=most_common)
        else:
            remaining_attributes = [a for a in attributes if a != best_attribute]
            tree.children[value] = c45(subset, remaining_attributes, target_attribute, data, value)

    return tree

data = df.copy()

attributes = data.columns[1:-1]  
target_attribute = 'Decision'

tree = c45(data, attributes, target_attribute, None)

new_sample = pd.Series({'Outlook': 'Sunny', 'Temp': 85, 'Humidity': 85, 'Wind': 'Weak'})

def classify(tree, sample, parent_value=None):
    if tree.result is not None:
        return tree.result
    attribute_value = sample[tree.attribute]
    if attribute_value in tree.children:
        return classify(tree.children[attribute_value], sample, attribute_value)
    else:
        # If the attribute value is not found, use the parent's value if available
        if parent_value is not None:
            return classify(tree.children[parent_value], sample, parent_value)
        else:
            most_common = data[target_attribute].value_counts().idxmax()
            return most_common

result = classify(tree, new_sample)
print("Predicted decision:", result)


Predicted decision: No


In [60]:
#implement CART algorithm

In [105]:
def gini_impurity(data, target_attribute):
    value_counts = data[target_attribute].value_counts()
    total = len(data)
    impurity = 1
    for value_count in value_counts:
        probability = value_count / total
        impurity -= probability ** 2
    return impurity

def gini_impurity_gain(data, attribute, target_attribute):
    impurity_before = gini_impurity(data, target_attribute)
    values, counts = np.unique(data[attribute], return_counts=True)
    impurity_after = 0
    for value, count in zip(values, counts):
        subset = data[data[attribute] == value]
        impurity_after += (count / len(data)) * gini_impurity(subset, target_attribute)
    return impurity_before - impurity_after

def cart(data, attributes, target_attribute):
    if len(np.unique(data[target_attribute])) == 1:
        return Node(result=data[target_attribute].iloc[0])

    if len(attributes) == 0:
        most_common = data[target_attribute].value_counts().idxmax()
        return Node(result=most_common)

    best_attribute = max(attributes, key=lambda a: gini_impurity_gain(data, a, target_attribute))
    tree = Node(attribute=best_attribute)

    values = np.unique(data[best_attribute])
    for value in values:
        subset = data[data[best_attribute] == value]
        if len(subset) == 0:
            most_common = data[target_attribute].value_counts().idxmax()
            tree.children[value] = Node(result=most_common)
        else:
            remaining_attributes = [a for a in attributes if a != best_attribute]
            tree.children[value] = cart(subset, remaining_attributes, target_attribute)

    return tree

tree = cart(data, attributes, target_attribute)
"""
new_sample = pd.Series({'Outlook': 'Sunny', 'Temp': 69, 'Humidity': 80, 'Wind': 'Strong'})

def classify(tree, sample):
    if tree.result is not None:
        return tree.result
    attribute_value = sample[tree.attribute]
    if attribute_value in tree.children:
        return classify(tree.children[attribute_value], sample) """

result = classify(tree, new_sample)
print("Predicted decision:", result)

Predicted decision: Yes


In [71]:
print(df.iloc[:, :-1])
print(df.iloc[:, -1])

    Day   Outlook  Temp  Humidity    Wind
0     1     Sunny    85        85    Weak
1     2     Sunny    80        90  Strong
2     3  Overcast    83        78    Weak
3     4      Rain    70        96    Weak
4     5      Rain    68        80    Weak
5     6      Rain    65        70  Strong
6     7  Overcast    64        65  Strong
7     8     Sunny    72        95    Weak
8     9     Sunny    69        70    Weak
9    10      Rain    75        80    Weak
10   11     Sunny    75        70  Strong
11   12  Overcast    72        90  Strong
12   13  Overcast    81        75    Weak
13   14      Rain    71        80  Strong
0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: Decision, dtype: object


In [96]:
"""d= df.to_numpy()
x = d[:, :]
y = d[:, -1]

print(x)"""

'd= df.to_numpy()\nx = d[:, :]\ny = d[:, -1]\n\nprint(x)'